In [1]:
from crvusdsim.pool import get

ETH_POOL = "weth"
sim_market = get(ETH_POOL, bands_data="controller")
pool = sim_market.pool
stableswap_pools = sim_market.stableswap_pools

In [2]:
# Verify snapshot context works on LLAMMA pools
i, j = 0, 1
high = pool.get_max_trade_size(i, j)

prev_bals = (sum(pool.bands_x.values()), sum(pool.bands_y.values()))

print("Pre Trade")
print("Balances", prev_bals)
print("High", high)
print("Stablecoin", pool.BORROWED_TOKEN.balanceOf[pool.address])
print("Collateral", pool.COLLATERAL_TOKEN.balanceOf[pool.address])
print()

with pool.use_snapshot_context():
    amt_in, amt_out, _ = pool.trade(i, j, high)
    swap_bals = (sum(pool.bands_x.values()), sum(pool.bands_y.values()))
    print("Amount in", high)
    print("Amount out", amt_out)
    print()
    print("Post Trade")
    print("Balances", swap_bals)
    print("Stablecoin", pool.BORROWED_TOKEN.balanceOf[pool.address])
    print("Collateral", pool.COLLATERAL_TOKEN.balanceOf[pool.address])
    print()

new_bals = (sum(pool.bands_x.values()), sum(pool.bands_y.values()))
print("After reversing snapshot context")
print("Balances", new_bals)
print("Stablecoin", pool.BORROWED_TOKEN.balanceOf[pool.address])
print("Collateral", pool.COLLATERAL_TOKEN.balanceOf[pool.address])

assert prev_bals == new_bals, (prev_bals, new_bals)
assert swap_bals != new_bals

Pre Trade
Balances (1343680871414493832282112, 14456000806805394703476)
High 39685409476929496574424390
Stablecoin 1343680871414493832282112
Collateral 14456000806805394703476

Amount in 39685409476929496574424390
Amount out 11515479576562433339391

Post Trade
Balances (41029090348343990406468413, 2940521230242961364085)
Stablecoin 41029090348343990406706502
Collateral 2940521230242961364085

After reversing snapshot context
Balances (1343680871414493832282112, 14456000806805394703476)
Stablecoin 1343680871414493832282112
Collateral 14456000806805394703476


In [3]:
# Verify snapshot context works on StableSwap pools from crvusdsim
i, j = 1, 0

stableswap_pool = stableswap_pools[2]
high = stableswap_pool.get_max_trade_size(i, j)
prev_bals = stableswap_pool.balances.copy()

print("Pre Trade")
print("Balances", prev_bals)
print("High", high)
print("Allowance", stableswap_pool.coins[0].balanceOf[stableswap_pool.address])
print()

with stableswap_pool.use_snapshot_context():
    amt_out = stableswap_pool.trade(i, j, high)
    swap_bals = stableswap_pool.balances.copy()
    print("Post Trade")
    print("Amount out", amt_out)
    print("Balances", swap_bals)
    print("Allowance", stableswap_pool.coins[0].balanceOf[stableswap_pool.address])
    print()

new_bals = stableswap_pool.balances.copy()
print("After reversing snapshot context")
print("Balances", new_bals)
print("Allowance", stableswap_pool.coins[0].balanceOf[stableswap_pool.address])

assert prev_bals == new_bals, (prev_bals, new_bals)
assert swap_bals != new_bals

Pre Trade
Balances [17626578665113, 19319573233956232808061670]
High 19266994687812898330266888
Allowance 17626578665113

Post Trade
Amount out (19266994687812898330266888, 17448567847174, 1745031287846187000545)
Balances [177138302296, 38586567921769131138328558]
Allowance 178010817939

After reversing snapshot context
Balances [17626578665113, 19319573233956232808061670]
Allowance 17626578665113
